## Fast Python3 For Beginners

### Multithreading
> 启动一个线程就是把一个函数传入并创建Thread实例，然后调用start()开始执行：

In [1]:
import time, threading

In [2]:
def loop():
    print("Threading %s is running..." % threading.current_thread().name)
    n = 0
    while n < 5:
        n += 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended' % (threading.current_thread().name))
    
print("Threading %s is running..." % (threading.current_thread().name))
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

Threading MainThread is running...
Threading LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread endedthread MainThread ended.



> 由于任何进程默认就会启动一个线程，我们把该线程称为**主线程**，主线程又可以启动新的线程，Python的threading模块有个current_thread()函数，它永远返回当前线程的实例。主线程实例的名字叫**MainThread**，子线程的名字在创建时指定，我们用LoopThread命名子线程。名字仅仅在打印时用来显示，完全没有其他意义，如果不起名字Python就自动给线程命名为Thread-1，Thread-2……

### Lock
> 多线程和多进程最大的不同在于，多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响，而多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改，因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。

In [3]:
import threading as th

balance = 0

def change_it(n):
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(10000000):
        change_it(n)
        
t1 = th.Thread(target=change_it, args=(5,))
t2 = th.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


In [4]:
import threading as th

balance = 0
lock = th.Lock()

def change_it(n):
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(10000000):
        lock.acquire()
        try:
            # 放心地改吧:
            change_it(n)
        finally:
            # 改完了一定要释放锁:
            lock.release()
            

t1 = th.Thread(target=change_it, args=(5,))
t2 = th.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)      


0


### ThreadLocal
> 在多线程环境下，每个线程都有自己的数据。一个线程使用自己的局部变量比使用全局变量好，因为局部变量只有线程自己能看见，不会影响其他线程，而全局变量的修改必须加锁。

In [5]:
import threading as th

# 创建全局Threading对象
local_school = th.local()

def process_student():
    std = local_school.student
    print("Hello, %s (in %s)" % (std, th.current_thread().name))
    
def process_thread(name):
    # 绑定ThreadLocal的student:
    local_school.student = name
    process_student()

t1 = th.Thread(target=process_thread, args=('Alice',), name='Thread-A')
t2 = th.Thread(target=process_thread, args=('Bob',), name='Thread-B')
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)
Hello, Bob (in Thread-B)
